In [1]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score

from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

In [ ]:
best_gradBoost = pickle.load(open('curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('curr_models/xgBoost.pkl', 'rb'))
kmer = 4

s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

pred_arr = []
xg_pred = []
for ind, file in enumerate(os.listdir("./virome_contigs")):
    fasta_sequences = SeqIO.parse(open(f"./virome_contigs/{file}"),'fasta')

    fasta = [x for x in fasta_sequences][0]
    # print(fasta)
    
    name, sequence = fasta.id, str(fasta.seq)
    X_info = sequence.lower()

    oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    cols_when_model_builds = best_gradBoost.feature_names_in_
    kmer_df=kmer_df[cols_when_model_builds]
    
    pred_arr.append(best_gradBoost.predict(kmer_df))
    
    cols_when_model_builds = best_xgBoost.get_booster().feature_names
    kmer_df=kmer_df[cols_when_model_builds]

    xg_pred.append(best_xgBoost.predict(kmer_df))


    
pred_arr = np.asarray(pred_arr)
xg_pred = np.asarray(xg_pred)


    # print(best_gradBoost.predict(kmer_df), sequences.loc[ind]['isZoonotic'])
    # print(best_gradBoost.predict(kmer_df), sequences['isZoonotic'])
        # print(accuracy_score())

      accession                                           sequence  isZoonotic
0   NC_025403.1  accagagggaaaatataacaatgtcgttttatagcgatgtaaataa...           1
1   NC_025404.1  accagagggaaaattaagaaaggtcgttccaagacgacttaaaaga...           1
2   NC_028246.1  acggagaaaaacaaaaaaactatagtgattagataaataaggaaaa...           1
3   NC_002077.1  ttgcccactccctctctgcgcgctcgctcgctcggtggggcctgcg...           1
4   NC_006152.1  ctctcccccctgtcgcgttcgctcgctcgctggctcgtttgggggg...           1
..          ...                                                ...         ...
95  NC_004218.1  gtattaaatttttgtaagtcgttatggaattatttagtgacagtgg...           1
96  NC_004219.1  gtatttaaaattcatgtttttgcatcatggcgtgggttacgcaagc...           1
97  NC_004220.1  gtattaaaaattacaagaacctaacattgcaatggagatcttgaga...           1
98  NC_004221.1  gtatttaaaattatagaaagttctgaacctaggggtctttctgtct...           1
99  NC_004204.1  gtattaaaattcagcaattgtccaatttaggaaacattctgtttaa...           1

[100 rows x 3 columns]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [ ]:
print(sequence['isZoonotic'])